# Task 5.2 Understand Theortical Background behind Sampler

## Objective 1: Get Started with Primitives

Check Task 5.1 objective 1

## Objective 2: Sampler


### Attributes

* mode
* options
* version


### Methods

* backend()
* run()